# Approaches to send messages to broker

## 1) Fire and forget
- **Profile**: We do not wait for response
- **Usage**: Cases when it is okay if we loose somw messages(3-5%. Don't use it if we cannot loose any messages
- **Drawbacks**: Loose of messages and less control over the flow

## 2) Synchronous send
- **Profile)**: We are waiting for response from Kafka. If we get success then it returns record_metadata, otherwise we get an exception
- **Usage**: When all the messages are critical and we cannot afford to loose anything
- **Drawbacks**: It slows down the program because we need to wait for the response(acknowledgement)

## 3) Asynchronous send
- **Profile)**: We don't wait for success or failure. We use callbacks functions
- **Usage**: When we need fast sending and at the same time exception handling
- **Drawbacks**: We have limited max.in.flight requests(the amount of messages we can send to kafka without receiving responses)